# <center>Supervised Learning - Text Classification</center>
References:
* http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

## 1. What we'll cover in this lecture
* Review basic concepts of machine learning
  * Cross validation
  * Performance metrics: recall and precision
* Text Classification  
  * Assign a document into one  or more pre-defined categories (or labels)
    * Input: 
      - a document $d$ 
      - a fixed set of classes C = {$c_1$, $c_2$,..., $c_J$}
      - A training set of $m$ hand-labeled documents ($d_1,c_1$),....,($d_m,c_m$)
    * Output: a classifier that predicts $d$ to some classes $c$ $\subset$ C
  * **Single-label** classification: e.g. spam dection, sentiment detection
  * **Multi-label** classification: e.g. news categorization

## 2. Review basic concepts of machine learning
### 2.1. Cross validation 
  * Defition: model valiation technique for assessing how a model will generalize to an independent data set
  * *k*-fold cross validation: 
    - Often, there is not enough data to allow some of it to be kept back for testing. 
    - Thus, the data is separated into k subsets. Each time, one of the subsets is held as the test set (a.k.a holdout) and the rest of them is used as the training set. 
    - This method repeats *k* times and each time with a different subset as the test set. 
  <img src="cross_validation.png" width="40%"> [source] (http://spark-public.s3.amazonaws.com/nlp/slides/sentiment.pptx)

### 2.2. Performance metrics
  * Precision: precentage of true cases among the predicated true cases
  * Recall:  precentage of true cases that have been retrieved over the total number of true cases
  * F-score: $$\frac{2*precision*recall}{precision+recall}$$
  * Example: 
Confusion Matrix: <img src="confusion_matrix.png">
    * For "YES" group: 
      - precision=?, 
      - recall=?, 
      - f-score=?
      <img src="precision_recall.png" width="60%">
    * For "NO" group:
      - precision=?, 
      - recall=?, 
      - f-score=?
  * Overall model performance
    * precision_macro (or recall_macro or f1_macro) is calculated as:
      1. calculate precision for each label
      2. average over labels 
    * precision_micro (or recall_micro or f1_micro): calculates metrics globally regardless of labels
    * With inbalanced classes, the difference between these two metrics may be significant

## 3. Text Classification

* Basic process
  1. Load and preprocess sample data
  2. Extract features: e.g. bag of words with TF-IDF weights
  3. Split data into trainning and test sets following cross validation method
  4. Train a classifier/model with the training dataset using selected classification algorithm for each fold
  5. Calculate performance
 
* Considerations for deciding text classification algorithms
  - should be effective in high dimensional spaces
  - should be effective even if the number of features is greater than the number of samples
    * Is regression a good alogorithm if you have a small number of text samples?
  - some good algorithms to start with:
      - Naive Bayes (https://web.stanford.edu/class/cs124/lec/naivebayes.pdf): baseline for performance benchmarking of text classification algorithms
      - Support Vector Machine (SVM) (https://www.svm-tutorial.com/2014/11/svm-understanding-math-part-1/)
  

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Exercise 3.1.: Load data 
# Load datasets (http://qwone.com/~jason/20Newsgroups/)
# For convenience, a subset of the data has been saved into "twenty_news_data.pkl"

import pickle
data=pickle.load(open("twenty_news_data.pkl","rb"))

# data is a list of tuple (text, label)
# print out the first document
print("Text: "+data[0][0]+"\nGroup: "+data[0][1])

In [ ]:
# Exercise 3.2. Prepare text and class labels

# split data as list of tuples into text and target tuples
text,target=zip(*data)

# convert tuple to list
text=list(text)
target=list(target)

## 3.1. TF-IDF matrix generation
- Function: **sklearn.feature_extraction.text.TfidfVectorizer**(input='content',encoding='utf-8', decode_error='strict', token_pattern='(?u)\b\w\w+\b', lowercase=True, stop_words=None, ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, norm='l2', use_idf=True, smooth_idf=True, ...)
- Some useful parameters:
    * **input** : string {'filename', 'file', 'content').
    * **encoding** : string, 'utf-8' by default.
If bytes or files are given to analyze, this encoding is used to decode.
    * **decode_error** : {'strict', 'ignore', 'replace'}: Instruction on what to do if a byte sequence is given to analyze that contains characters not of the given encoding. By default, it is 'strict', meaning that a UnicodeDecodeError will be raised. Other values are ‘ignore’ and ‘replace’.
    * **token_pattern** : Regular expression denoting what constitutes a “token”. The default is '(?u)\b\w\w+\b', i.e. a token contains at least two word characters in unicode (note: ?u: unicode, \b: space or non-word character, i.e. boundary, \w: word character). 
    * **ngram_range** : tuple (min_n, max_n): The lower and upper boundary of the range of n-values for different n-grams to be extracted. 
    * **stop_words** : string {‘english’}, list, or None (default)
    * **lowercase** : boolean, default True: Convert all characters to lowercase before tokenizing.
    * **max_df/min_df** : float in range [0.0, 1.0] or int, default=1.0: When building the vocabulary ignore terms that have a document frequency strictly higher (lower) than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. 
    * **max_features** : int or None, default=None. If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
    * **norm** : 'l1', 'l2' or None, optional. Norm used to normalize term vectors. None for no normalization.
    * **use_idf** : boolean, default=True. Enable inverse-document-frequency reweighting.
    * **smooth_idf** : boolean, default=True. Smooth idf weights by adding one to document frequencies, as if an extra document was seen containing every term in the collection exactly once. Prevents zero divisions.
- For all the parameters, see http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


In [ ]:
# Exercise 3.3. Create TF-IDF Matrix

from sklearn.feature_extraction.text import TfidfVectorizer

# initialize the TfidfVectorizer 

tfidf_vect = TfidfVectorizer() 

# with stop words removed
#tfidf_vect = TfidfVectorizer(stop_words="english") 

# generate tfidf matrix
dtm= tfidf_vect.fit_transform(text)

print("type of dtm:", type(dtm))
print("size of tfidf matrix:", dtm.shape)

In [ ]:
# Exercise 3.4. Examine TF-IDF

# 1. Check vocabulary
# Vocabulary is a dictionary mapping a word to an index
print("type of vocabulary:", type(tfidf_vect.vocabulary_))
print("index of word 'city' in vocabulary:", \
      tfidf_vect.vocabulary_['city'])


# get mapping from word index to word
# i.e. reversal mapping of tfidf_vect.vocabulary_
voc_lookup={tfidf_vect.vocabulary_[word]:word \
            for word in tfidf_vect.vocabulary_}

# the number of words in the vocabulary
print("total number of words:", len(voc_lookup ))


# 2. check words with top tf-idf wights in a document, e.g. 1st document

print("\nOriginal text: \n"+text[0])

print("\ntfidf weights: \n")

# first, covert the sparse matrix row to a dense array
doc0=dtm[0].toarray()[0]
print(doc0.shape)

# get index of top 20 words
top_words=(doc0.argsort())[::-1][0:20]
print([(voc_lookup[i], doc0[i]) for i in top_words])


In [ ]:
# Exercise 3.5. classification using a single fold

# use MultinomialNB algorithm
from sklearn.naive_bayes import MultinomialNB

# import method for split train/test data set
from sklearn.model_selection import train_test_split

# import method to calculate metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

# split dataset into train (70%) and test sets (30%)
X_train, X_test, y_train, y_test = train_test_split(\
                dtm, target, test_size=0.3, random_state=0)

# train a multinomial naive Bayes model using the testing data
clf = MultinomialNB().fit(X_train, y_train)

# predict the news group for the test dataset
predicted=clf.predict(X_test)

# get the list of unique labels
labels=sorted(list(set(target)))

# calculate performance metrics. 
# Support is the number of occurrences of each label

precision, recall, fscore, support=\
     precision_recall_fscore_support(\
     y_test, predicted, labels=labels)

print(labels)
print(precision)
print(recall)
print(fscore)
print(support)

# another way to get all performance metrics
print(classification_report\
      (y_test, predicted, target_names=labels))

In [ ]:
# Exercise 3.6.  predict new documents

docs_new = ['God is love', 'OpenGL on the GPU is fast']

# generate tifid for new documents
X_new_tfidf = tfidf_vect.transform(docs_new)

print(X_new_tfidf.shape)

# predict classes for new documents
predicted = clf.predict(X_new_tfidf)

for idx, doc in enumerate(docs_new):
    print('%r => %s' % (doc, predicted[idx]))


In [ ]:
# Exercise 3.7. Classification with stop words removed
# Can removing stop words improves performance?
# In Exercise 3.3, uncomment line 10 and comment line 7
# Run Exercise 3.3
# Run Exercise 3.5

In [ ]:
# Exercise 3.8. Run 5 fold cross validation
# to show the generalizability of the model

# import cross validation method
from sklearn.model_selection import cross_validate
#from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import MultinomialNB

metrics = ['precision_macro', 'recall_macro', "f1_macro"]

#clf = MultinomialNB()
clf = MultinomialNB(alpha=0.8)

cv = cross_validate(clf, dtm, target, scoring=metrics, cv=5)
print("Test data set average precision:")
print(cv['test_precision_macro'])
print("\nTest data set average recall:")
print(cv['test_recall_macro'])
print("\nTest data set average fscore:")
print(cv['test_f1_macro'])

# To see the performance of training data set use 
# cv['train_xx_macro']
print(cv['train_f1_macro'])

# The metrics are quite stable across folds.
# The performance between training and test sets is small
# This indicates the model has good generalizability

In [ ]:
# Exercise 3.9. Multinominal NB 
# with different smoothing parameter alpha
# comment line 11 and uncomment 12 in Exercise 3.8
# use different alpha value to see if it affects performance

In [ ]:
# Exercise 3.10. SVM model

from sklearn.model_selection import cross_validate
#from sklearn.metrics import precision_recall_fscore_support
from sklearn import svm

metrics = ['precision_macro', 'recall_macro', "f1_macro"]

# initiate an linear SVM model
clf = svm.LinearSVC()

cv = cross_validate(clf, dtm, target, scoring=metrics, cv=5)
print("Test data set average precision:")
print(cv['test_precision_macro'])
print("\nTest data set average recall:")
print(cv['test_recall_macro'])
print("\nTest data set average fscore:")
print(cv['test_f1_macro'])


## 3.3. Parameter tuning using grid search
* Each classification model has a few parameters
  * e.g. "stop_words": "english" or None, min_df: [1,2,3, ...]
  * e.g. MultinomialNB(alpha=1.0)
  * e.g. LinearSVC(penalty=’l2’, loss=’squared_hinge’,...)
* Instead of tweaking the parameters of the various components, it is possible to run an exhaustive search of the best parameters on a grid of possible values

In [ ]:
# Exercise 3.3.1 Grid search

# import pipeline class
from sklearn.pipeline import Pipeline

# import GridSearch
from sklearn.model_selection import GridSearchCV

# build a pipeline which does two steps all together:
# (1) generate tfidf, and (2) train classifier
# each step is named, i.e. "tfidf", "clf"

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB())
                   ])

# set the range of parameters to be tuned
# each parameter is defined as 
# <step name>__<parameter name in step>
# e.g. min_df is a parameter of TfidfVectorizer()
# "tfidf" is the name for TfidfVectorizer()
# therefore, 'tfidf__min_df' is the parameter in grid search

parameters = {'tfidf__min_df':[2,5,10],
              'tfidf__stop_words':[None,"english"],
              'clf__alpha': [0.5,1.0,2.0],
}

# the metric used to select the best parameters
metric =  "f1_macro"

# GridSearch also uses cross validation
gs_clf = GridSearchCV\
(text_clf, param_grid=parameters, scoring=metric, cv=5)

# due to data volume and large parameter combinations
# it may take long time to search for optimal parameter combination
# you can use a subset of data to test
gs_clf = gs_clf.fit(text, target)


In [ ]:
# gs_clf.best_params_ returns a dictionary 
# with parameter and its best value as an entry

for param_name in gs_clf.best_params_:
    print(param_name,": ",gs_clf.best_params_[param_name])

print("best f1 score:", gs_clf.best_score_)

In [ ]:
# Exercise 3.3.2 Grid search
# Modify Exercise 3.3 and Exercise 3.8 
# to use the best parameters found
# re-create the Multinominal NB classifier

# also, check the dimension reduction of feature space by set min_df to 2

## 4. Multi-label classification
- So far we only cover single-label classification, i.e. assign one class to each sample
- Multilabel classification emerges as a challenging problem, where classes are not mutually exclusive 
  * music categorization 
  * semantic classification of images
  * tagging
- **One-Vs-the-Rest** Strategy (a.k.a **one-vs-all**)
  * fitting one classifier per class. For each classifier, the class is fitted against all the other classes.
  * for $n$ classes (labels), $n$ classifier is needed
  * Advantage: good interpretability - Since each class is represented by one and only one classifier, it is possible to gain knowledge about the class by inspecting its corresponding classifier
  * Disadvantage: 
     * many classifiers are created if there is a large number classes
     * ignore the structure (or dependencies) of classes
- **Class indication matrix** (or **one-hot encoding**): Encode categorical integer features using a one-hot aka one-of-K scheme. 

| Document    | Money       | Investment | Crime & Justice |
| :-----------|:-----------:|:----------:|:--------------:|
| 1           | 0           |      0     | 1              |
| 2           | 1           |      1     | 0              |
| 3           | 1           |      0     | 0              |
| 4           | 0           |      1     | 1              |

- **dataset**: Yahoo News Ranked Multilabel Learning dataset (http://research.yahoo.com)
  - A subset is selected
  - 4 classes, 6426 samples

In [ ]:
# Exercise 4.1 Multi-label classification- Load data

import json
data=json.load(open("ydata.json","r"))

docs,labels=zip(*data)

# show sample examples
docs[1]
labels[1]


In [ ]:
# Exercise 4.2 One-hot coding of classes

from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(labels)
# check size of indicator matrix
Y.shape
# check classes
mlb.classes_

# check # of samples in each class
np.sum(Y, axis=0)

In [ ]:

# Exercise 4.3 Multi-label classification- one vs. rest classifier

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# split dataset into train (70%) and test sets (30%)
X_train, X_test, Y_train, Y_test = train_test_split(\
                docs, Y, test_size=0.3, random_state=0)



classifier = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words="english", min_df=2)),
    ('clf', OneVsRestClassifier(LinearSVC()))])

classifier.fit(X_train, Y_train)



In [ ]:
# Exercise 4.4 Multi-label classification- Performance report

from sklearn.metrics import classification_report

predicted = classifier.predict(X_test)

predicted.shape
predicted[0:2]
Y_test[0:2]

print(classification_report\
      (Y_test, predicted, target_names=mlb.classes_))